In [ ]:
# Initial development for DicomFile

# Import classes and methods
from pybrors.dicom import DicomFile

# Create file instance
dicom_file = DicomFile(file_path="../data/dicom/dicom_img_file.dcm")
print(dicom_file)
print(dicom_file.dataset["PatientName"].value)

# Anonymize DICOM file
if dicom_file.anonymize():
    print("DICOM has been anonymized.")
else:
    print("DICOM has not been anonymized.")

# Display anonymized file information
print(dicom_file)
print(dicom_file.dataset["PatientName"])

In [ ]:
# Initial development for DicomDir

# Import packages and submodules
import time

# Import classes and methods
from pybrors.dicom import DicomDir

start_time = time.perf_counter()

# Create file instance
dicom_dir = DicomDir(dir_path="../data/dicom")

# Anonymize DICOM file
if dicom_dir.anonymize():
    print("DICOM directory has been anonymized.")
else:
    print("DICOM directory has not been anonymized.")

end_time   = time.perf_counter()
print("Runtime: %0.4f seconds" % (end_time - start_time))

In [ ]:
# Initial development of DicomData

# Import classes and methods
from pybrors.dicom import DicomData
from matplotlib import pyplot

# Create file data instance
dicom_file = DicomData(file_path="../data/dicom/dicom_sbi_file.dcm")
print(dicom_file.file_list)
print(dicom_file.file_dir)
print(dicom_file.dataset)
pyplot.imshow(dicom_file.data, interpolation="nearest", cmap="gray")

In [ ]:
# Anonymize DICOM directory using rust library

# Import packages and submodules
import os
import time

# Import classes and methods
from pybrors import rust_lib
from pybrors.dicom import DicomDir

# Anonymize DICOM directory
start_time = time.perf_counter()
dicom_dir        = DicomDir(dir_path="../data/dicom")
rust_lib.anonymize_dicomdir(
    dicom_dir.file_list, os.path.join(dicom_dir.dir_path, "anonymized"))
end_time   = time.perf_counter()
print("Runtime: %0.4f seconds" % (end_time - start_time))

In [ ]:
# Initial development of DicomData

# Import classes and methods
from pybrors.dicom import DicomData
from matplotlib import pyplot

# Create file data instance for a folder
dicom_dir = DicomData(dir_path="../data/dicom/dicom_exam/CT/FatFraction")
print(dicom_dir.file_list)
print(dicom_dir.file_dir)
print(dicom_dir.dataset)
pyplot.imshow(dicom_dir.data[:, :, 0], interpolation="nearest", cmap="gray")


In [ ]:
# Initial development for SbiFile

# Import classes and methods
from pybrors.dicom import SbiFile
from matplotlib import pyplot

# Create file instance
dicom_file = SbiFile(file_path="../data/dicom/dicom_sbi_file.dcm")
exp_maps = dicom_file.generate_exponential();

In [ ]:
from numpy import min, max
idx = 0
print(min(exp_maps[:,:,idx]))
print(max(exp_maps[:,:,idx]))
pyplot.imshow(exp_maps[:,:,idx], interpolation="nearest", cmap="jet")

In [ ]:
# Initial development for SbiFile

# Import classes and methods
from pybrors.dicom import SbiFile
from pybrors.dicom import DicomData
from matplotlib import pyplot

# Create file instance
dicom_file = SbiFile(file_path="../data/dicom/dicom_sbi_file.dcm")
dicom_file.export_exp_maps()


In [ ]:
dicom_file = DicomData(file_path="../data/dicom/dicom_sbi_file_ec.dcm")
pyplot.imshow(dicom_file.data, interpolation="nearest", cmap="jet")

In [ ]:
# Import libraries
import lmfit
from numpy import arange, exp, stack, zeros, min, max, std, mean, concatenate
from pybrors.dicom import SbiFile

# Define exponential model
def exp_model(energy, mu_0, mu_inf, e_c):
    return (mu_0 - mu_inf) * exp(-energy / e_c) + mu_inf

# Load SBI File
sbi_file = SbiFile(file_path="../data/dicom/dicom_sbi_file.dcm")

# Generate MonoE maps
energy_range = arange(40., 60., 1.)
energy_range = concatenate((energy_range, arange(60., 80., 2.)))
energy_range = concatenate((energy_range, arange(80., 120., 5.)))
energy_range = concatenate((energy_range, arange(120., 200., 10.)))
monoe_stack = [sbi_file.generate_monoe(int(energy)) for energy in energy_range]
monoe_stack = stack(monoe_stack, axis=2)

# Exponential fit voxel by voxel
exp_maps = zeros([monoe_stack.shape[0], monoe_stack.shape[1], 3])
std_threshold = max(monoe_stack) * 0.01
model = lmfit.Model(exp_model)
initial_guess = dict(mu_0=2000., mu_inf=200., e_c=20.)
for i in range(monoe_stack.shape[0]):
    for j in range(monoe_stack.shape[1]):
        tmp_std = std(monoe_stack[i,j,:])
        if tmp_std > std_threshold:
            results = model.fit(monoe_stack[i,j,:], energy=energy_range, **initial_guess)
            exp_maps[i,j,:] = [
                results.best_values['mu_0'],
                results.best_values['mu_inf'],
                results.best_values['e_c']
            ]
        else:
            tmp_mu = mean(monoe_stack[i,j,:])
            exp_maps[i,j,:] = [tmp_mu, tmp_mu, 0]


In [ ]:
# Import libraries
from matplotlib import pyplot

print(energy_range.shape)

# Visualize
idx = 0
print(min(exp_maps[:,:,idx]))
print(max(exp_maps[:,:,idx]))
pyplot.imshow(exp_maps[:,:,idx], interpolation="nearest", cmap="jet")